# Кредитная история.

---

In [767]:
import pandas as pd

In [768]:
df = pd.read_csv('credit_history.csv',parse_dates=['status date','date of loan issue'],dayfirst=True)

In [769]:
df.head(3)

,type of credit institution,сredit institution name,date of loan issue,loan amount(RUB),loan repayment amount(RUB),loan amount owed(RUB),loan status,status date
0,Микрокредит,"ООО МКК """"СОЮЗ 5""""""",2021-06-15,12440,27296,0,Счет закрыт,2022-03-10
1,Микрокредит,"ООО МКК """"Микроклад""""""",2021-06-01,4000,0,10000,Просрочен,2022-10-02
2,Потребит.кредит,"ООО МФК """"ЧЕСТНОЕ СЛОВО""""""",2021-05-22,35000,5715,82001,Просрочен,2022-05-18


---

## Обработка, очистка.

In [770]:
# присваеваем имена атрибутам, как в задании.
df.columns = ['тип кредитной организации', 'название организации', 'дата выдачи кредита', 'сумма выдачи',
            'сумма погашенного', 'сумма задолженности', 'статус кредита', 'дата статуса']
df.columns = df.columns.str.replace(' ','_').str.lower()

In [771]:
# приводим к нижнему регистру атрибуты  'тип_кредитной_организации' и 'статус_кредита'
df['тип_кредитной_организации'] = df['тип_кредитной_организации'].str.lower()

In [772]:
df['статус_кредита'] = df['статус_кредита'].str.lower()

In [773]:
# присутсвует избыточность в столбце 'название_организации'
df['название_организации'].value_counts()

ООО МФК ""Займер"""                   46
ООО МФК ""Мани Мен"""                 35
ООО МКК ""СМСФИНАНС"""                22
ООО «МИКРОКРЕДИТНАЯ                   18
ООО МКК ""Микроклад"""                11
ООО МКК ТУРБОЗАЙМ                     11
ООО «МФК «ЮПИТЕР 6»                   11
ООО «Киберлэндинг»                    10
ООО МФК ""ЧЕСТНОЕ СЛОВО"""            10
МФК ""Лайм-Займ"" (ООО)"               9
ООО МКК ""ДЗП-Центр"""                 7
ООО МКК ""Срочноденьги"""              4
ООО МКК ""Джет Мани Микрофинанс"""     4
ООО МФК ""Экофинанс"""                 3
ООО МКК ЗАЙМ-ЭКСПРЕСС                  3
ООО МКК ""СОЮЗ 5"""                    3
АО КБ ""Пойдём!"""                     2
ООО МКК ""ФИНТЕРРА"""                  2
ООО «Финанби»                          2
ООО ""ПрофиМани"""                     2
ООО МФК ""ЗАЙМ ОНЛАЙН"""               2
ООО МКК ""Смсфинанс"""                 2
ООО ""ФЖК"""                           1
ООО ""Займиго МФК"""                   1
ООО «Микрокредит

In [774]:
# срезами выделяем название организаций
def clean_column(column):
    if '""' in column:
         return column[column.find('""',1)+2:column.rfind('""')]
    elif '«' in column:
         return column[column.find('«',1)+1:]  
    elif '"' or '«' not in column:
        return  column[column.rfind(' ')+1:]
    return column       

In [775]:
# присваиваем новые значения , применяем методы str для окончательной очистки и приводим к нижнему регистру
df['название_организации'] = df['название_организации'].apply(clean_column).str.replace('«', '').str.replace('»','').str.replace('"','').str.lower().str.replace('мфк', '').str.lstrip()

In [776]:
# переименовываем строку 'потребит.кредит' в 'потребительский кредит'
df['тип_кредитной_организации'] = df['тип_кредитной_организации'].str.replace('потребит.кредит', 'потребительский кредит', regex=True)

In [777]:
# дубликаты отсутствуют
df.duplicated().sum()

0

In [778]:
# значений Null отсутствуют
df.isnull().sum()

тип_кредитной_организации    0
название_организации         0
дата_выдачи_кредита          0
сумма_выдачи                 0
сумма_погашенного            0
сумма_задолженности          0
статус_кредита               0
дата_статуса                 0
dtype: int64

---

## Анализ кредитной истории

In [779]:
df.head(3)

,тип_кредитной_организации,название_организации,дата_выдачи_кредита,сумма_выдачи,сумма_погашенного,сумма_задолженности,статус_кредита,дата_статуса
0,микрокредит,союз 5,2021-06-15,12440,27296,0,счет закрыт,2022-03-10
1,микрокредит,микроклад,2021-06-01,4000,0,10000,просрочен,2022-10-02
2,потребительский кредит,честное слово,2021-05-22,35000,5715,82001,просрочен,2022-05-18


- **всего 226 кредитных договоров**

In [780]:
len(df)

226

- **220 микрокредитов**
- **4 потребительских кредита**
- **2 кредитные карты**

In [781]:
df.groupby('тип_кредитной_организации')['сумма_выдачи'].count()

тип_кредитной_организации
кредитная карта             2
микрокредит               220
потребительский кредит      4
Name: сумма_выдачи, dtype: int64

- **общая сумма выданных средств 1.704.636 тысяч рублей**
- **общая сумма выплаченых средств 2.105.605 тысяч рублей**
- **общая сумма задолженности 101.140 тысяч рублей**

In [782]:
df['сумма_выдачи'].sum(), df['сумма_погашенного'].sum(), df['сумма_задолженности'].sum()

(1704636, 2105605, 101140)

In [783]:
table = df.groupby('статус_кредита', sort=False).sum(numeric_only=True)
table.sort_values('сумма_выдачи',ascending=False)

,сумма_выдачи,сумма_погашенного,сумма_задолженности
статус_кредита,,,
счет закрыт,1658636,2099890,0
просрочен,39000,5715,92001
передача данных прекращена,7000,0,9139


**количество договоров по статусам :**
   - **передача данных прекращена 1**
   - **просрочен 2** 
   - **счет закрыт 223**
    
**Отношение 3 к 223 или 0,013% задолженности к погашенным кредитам**

In [784]:
 df.groupby('статус_кредита')['сумма_выдачи'].count()

статус_кредита
передача данных прекращена      1
просрочен                       2
счет закрыт                   223
Name: сумма_выдачи, dtype: int64

In [785]:
df[df['статус_кредита'] == 'просрочен']

,тип_кредитной_организации,название_организации,дата_выдачи_кредита,сумма_выдачи,сумма_погашенного,сумма_задолженности,статус_кредита,дата_статуса
1,микрокредит,микроклад,2021-06-01,4000,0,10000,просрочен,2022-10-02
2,потребительский кредит,честное слово,2021-05-22,35000,5715,82001,просрочен,2022-05-18



   - **минимальная выданная сумма 500 рублей**
   - **максимальная выданная сумма 35000 тысяч рублей**
   - **средняя выданная сумма 7610 тысяч рублей**
   - **медиана 5000 тысяч рублей**
    

In [786]:
df[df['сумма_выдачи'] == 0]

,тип_кредитной_организации,название_организации,дата_выдачи_кредита,сумма_выдачи,сумма_погашенного,сумма_задолженности,статус_кредита,дата_статуса
61,кредитная карта,пойдём!,2021-02-05,0,78275,0,счет закрыт,2021-06-10
62,кредитная карта,пойдём!,2021-02-05,0,12499,0,счет закрыт,2021-07-09


In [787]:
df1 = df[df['сумма_выдачи'] != 0]

In [788]:
round(df1.agg({'сумма_выдачи': ['max','min','median','mean']}))

,сумма_выдачи
max,35000.0
min,500.0
median,5000.0
mean,7610.0


In [792]:
df.to_csv(r"E:\Users\X\Desktop\parsing_nbki1\transform\credit_history_final.csv", index=False) 